### Llama-3.2-3B-Instruct (fine-tuning)

In [1]:
%%capture
!pip install unsloth "xformers==0.0.28.post2"
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [46]:
from unsloth import FastLanguageModel

In [47]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.11.6: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [48]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

### Data Transformation done

In [101]:
import pandas as pd
from datasets import Dataset
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

# Step 1: Define and apply the chat template
# Ensure tokenizer is defined (replace with the appropriate model's tokenizer)
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

# Step 2: Load CSV and prepare data in the expected format
df = pd.read_csv("/content/final_data.csv")
data = []


system_prompt = """
You are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.

For any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.

However, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:

'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.'

Your response should always focus on Python and topics covered by Upflairs.
"""
for i in range(df.shape[0]):
    sample = {
        'conversations': [
            {"from": "system", "value": system_prompt},
            {'from': 'human', 'value': df.loc[i, "Question"]},
            {'from': 'gpt', 'value': df.loc[i, "Answer"]}
        ]
    }
    data.append(sample)

print("No. of samples in the dataset:", len(data))

# Step 3: Convert list of dictionaries to a Dataset object
dataset = Dataset.from_list(data)

# Step 4: Apply the standardization function
dataset = standardize_sharegpt(dataset)

# Step 5: Define the formatting function to apply the chat template
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        for convo in convos
    ]
    return {"text": texts}

# Step 6: Apply the formatting function with map
dataset = dataset.map(formatting_prompts_func, batched=True)

# Optional: View a sample of the formatted dataset
print(dataset[0])


No. of samples in the dataset: 39


Standardizing format:   0%|          | 0/39 [00:00<?, ? examples/s]

Map:   0%|          | 0/39 [00:00<?, ? examples/s]

{'conversations': [{'content': "\nYou are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\nFor any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.\n\nHowever, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:\n\n'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.'\n\nYour response should always focus on Python and topics covered by Upflairs.\n", 'role': 'system'}, {'content': 'tell me about upflairs?', 'role': 'user'}, {'content': "UpFlairs is an innovative educational technology company dedicated to empowering students across India. With a focus on emerging techn

In [102]:
print(dataset[2])

{'conversations': [{'content': "\nYou are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\nFor any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.\n\nHowever, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:\n\n'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.'\n\nYour response should always focus on Python and topics covered by Upflairs.\n", 'role': 'system'}, {'content': 'How can I enroll in a course?', 'role': 'user'}, {'content': "You can enroll in any course by visiting the Upflairs website, selecting the course you're interested in, and clicking the 'Enro

We look at how the conversations are structured for item 1:

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [103]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 15, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/39 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [104]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/39 [00:00<?, ? examples/s]

We verify masking is actually done:

In [105]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n\nYou are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\nFor any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.\n\nHowever, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:\n\n'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.'\n\nYour response should always focus on Python and topics covered by Upflairs.\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHow long is the internship program at Upflairs?<|eot_id|><|start_header_

In [106]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

"                                                                                                                                                                                      \n\nOur internship programs typically last 3-6 months, depending on the field and project complexity. You'll get hands-on experience and work with industry professionals.<|eot_id|>"

We can see the System and Instruction prompts are successfully masked!

In [107]:
trainer_stats = trainer.train()   # start the fine-tuning with epochs=1

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 39 | Num Epochs = 12
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,0.000300
2,0.000300
3,0.000200
4,0.000400
5,0.000200
6,0.004600
7,0.000600
8,0.005400
9,0.000300
10,0.000400


In [108]:
## ask the query after the training model
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "tell me about upflairs?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ntell me about upflairs?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nUpFlairs is an innovative educational technology company dedicated to empowering students across India. With a focus on emerging technologies like AI/ML, Data Science, Cloud computing, DevOps, Full Stack Web Development, Embedded Systems, IoT and Robotics. We've educated over 50K+ students worldwide, including those from prestigious institutions"]

In [109]:
## Ask a query after training the model (multiple questions)
from unsloth.chat_templates import get_chat_template

# Initialize the tokenizer with the chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

# Define a list of testing queries
test_queries = [
    {"role": "user", "content": "tell me about upflairs."},
    {"role": "user", "content": "What courses does Upflairs offer?"},
    {"role": "user", "content": "How do you use list comprehension at Upflairs to create a list of even numbers from 1 to 100?"},
    {"role": "user", "content": "write a program to print hello world."},
    {"role": "user", "content": "write a program to print hello world in python."},
    {"role": "user", "content": "write a program to print hello world in java?"},
    {"role": "user", "content": "how we can find the average of the integer array in c++?"},
    {"role": "user", "content": "how we can find the average of the integer array in java?"},
    {"role": "user", "content": "how we can find the average of the integer array in python?"}
]

# Process each query using the chat template and tokenize
for query in test_queries:
    inputs = tokenizer.apply_chat_template(
        [query],
        tokenize=True,
        add_generation_prompt=True,  # Must add for generation
        return_tensors="pt",
    ).to("cuda")

    # Generate responses with varied parameters for testing
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=64,
        use_cache=True,
        temperature=0.7,  # Lowered temperature for more deterministic responses
        min_p=0.1
    )

    # Decode and print each output
    response = tokenizer.batch_decode(outputs)
    print(f"Query: {query['content']}\nResponse: {response}")
    print("Robo response : ",response[0].split('\n')[-1][:-10]) # extracted exact response from the chat response
    print()


Query: tell me about upflairs.
Response: ["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ntell me about upflairs.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nUpFlairs is an innovative educational technology company dedicated to empowering students across India. With a focus on emerging technologies like AI/ML, Data Science, Cloud computing, DevOps, Full Stack Web Development, Embedded Systems, IoT and Robotics. We've educated over 50K+ students worldwide, including those from prestigious institutions"]
Robo response :  UpFlairs is an innovative educational technology company dedicated to empowering students across India. With a focus on emerging technologies like AI/ML, Data Science, Cloud computing, DevOps, Full Stack Web Development, Embedded Systems, IoT and Robotics. We've educated over 50K+ students worldwide, including those from p

In [113]:
from unsloth.chat_templates import get_chat_template

# Initialize the tokenizer with the chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

# Define a list of testing queries
test_queries = [
    {"role": "user", "content": "tell me about upflairs."},
    {"role": "user", "content": "What courses does Upflairs offer?"},
    {"role": "user", "content": "How do you use list comprehension at Upflairs to create a list of even numbers from 1 to 100?"},
    {"role": "user", "content": "write a program to print hello world."},
    {"role": "user", "content": "write a program to print hello world in python."},
    {"role": "user", "content": "write a program to print hello world in java?"},
    {"role": "user", "content": "how we can find the average of the integer array in c++?"},
    {"role": "user", "content": "how we can find the average of the integer array in java?"},
    {"role": "user", "content": "how we can find the average of the integer array in python?"}
]

# Open the file for appending responses
with open('response.txt', 'a+') as file:
    # Process each query using the chat template and tokenize
    for query in test_queries:
        inputs = tokenizer.apply_chat_template(
            [query],
            tokenize=True,
            add_generation_prompt=True,  # Must add for generation
            return_tensors="pt",
        ).to("cuda")

        # Generate responses with varied parameters for testing
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=64,
            use_cache=True,
            temperature=0.7,  # Lowered temperature for more deterministic responses
            min_p=0.1
        )

        # Decode and process the response
        response = tokenizer.batch_decode(outputs)
        clean_response = response[0].split('\n')[-1][:-10]  # Extract last part of the response

        # Write to the file with appropriate formatting
        file.write(f"Query: {query['content']}\n")
        file.write(f"Robo response: {clean_response}\n\n")

        # Also print to the console for debugging
        print(f"Query: {query['content']}")
        print("Robo response:", clean_response)
        print()


Query: tell me about upflairs.
Robo response: UpFlairs is an innovative educational technology company dedicated to empowering students across India. With a focus on emerging technologies like AI/ML, Data Science, Cloud computing, DevOps, Full Stack Web Development, Embedded Systems, IoT and Robotics. We've educated over 50K+ students worldwide, including those from prestigious in

Query: What courses does Upflairs offer?
Robo response: Upflairs offers courses in Data Science and Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding. We also provide internships to help you gain real-world experience.

Query: How do you use list comprehension at Upflairs to create a list of even numbers from 1 to 100?
Robo response: [x for x in range(1, 101) if x % 2 == 0]

Query: write a program to print hello world.
Robo response: print('Hello World')

Query: write a program to print hello world in python.
Robo response: print('Hello World')

Query: write a program to print hello

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The Fibonacci sequence is a series of numbers where each number is the sum of the two preceding numbers. 

The sequence you provided was: 1, 1, 2, 3, 5, 8, 13

The next number in the sequence would be 21, which is 8 + 13. The sequence continues as: 21, 34, 55, 89, 144, 233.<|eot_id|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The Eiffel Tower, located in the heart of Paris, stands tall among the city's historic and cultural landmarks. This iron structure, standing at an impressive 324 meters high, offers breathtaking views of the City of Light's iconic landscape. The Eiffel Tower was built for the 1889 World's Fair and has since become a symbol of French engineering and culture.<|eot_id|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )